<a href="https://colab.research.google.com/github/MasaoMori/Tokyo-Ward-Assembly-Election/blob/main/untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pathlib

cpath = os.getcwd()
ppath = str(pathlib.Path(cpath))
print(cpath, ppath)

/Users/mori/Documents/Devel/Tokyo_Ward_Assembly_Election/jupyter /Users/mori/Documents/Devel/Tokyo_Ward_Assembly_Election/jupyter


In [4]:
# 準備
import pandas as pd
import numpy as np

#pd.options.display.float_format = '{:,.0f}'.format

ele_dat = pd.read_excel(cpath + '/../data/区議会議員選挙過去データ.xlsx', 
                        sheet_name="選挙結果（生）",
                        dtype={'有権者数':int,'投票者数':float,'投票率%':float, '得票率%':float})
ele_par = pd.read_excel(cpath + '/../data/区議会議員選挙過去データ.xlsx', sheet_name="政党名")

ddcols = ['年', '区名', '当落', '得票数', '氏名', 'よみ', '年齢',
          '性別', '党派', '新旧', '主な肩書き','得票率%']
ele_dd = ele_dat[ddcols]

# 区名のコード

wards = pd.read_csv(cpath + '/../data/tokyo23ku.csv', encoding='Shift-JIS')
wards['区名']=wards['自治体名'].str.strip('区')
wards_master = wards[['自治体コード','区名']]

ele_result = pd.merge(pd.merge(ele_dd, ele_par, on="党派"), wards_master, on='区名')

In [5]:
print(list(ele_dat.columns))

['年', '区名', '当落', '得票数', '氏名', 'よみ', '年齢', '性別', '党派', '新旧', '主な肩書き', '投票日', '告知日', '定数', '候補者数', '有権者数', '投票率%', '投票者数', '得票率%']


データはこんな感じで出ています。１レコード、1人の立候補者のデータです。末尾のカラムには、選挙区での概要情報がつけてあります。

In [19]:
with pd.ExcelWriter('./区議会議員選挙過去データ.xlsx', mode='a') as writer:
    ele_result.to_excel(writer, sheet_name='選挙結果',index=False)
    wards_master.to_excel(writer, sheet_name='区名マスタ', index=False)


In [43]:
print(list(ele_result.columns))

['年', '区名', '当落', '得票数', '氏名', 'よみ', '年齢', '性別', '党派', '新旧', '主な肩書き', '投票日', '告知日', '定数', '候補者数', '有権者数', '投票率%', '投票者数', '得票率%', '略称党名', '党順序', '自治体コード']


In [5]:
###　1. 過去3回の区議会選挙における、有権者数と投票者数の推移
ele_summary = ['年','区名','自治体コード',
               '投票日', '告知日', '定数', '候補者数',
               '有権者数', '投票率%', '投票者数', '得票率%']
voters_tmp = ele_result[ele_summary]
# 有権者数と投票者数を年、区別に抽出

voters_q = voters_tmp.groupby(['年','区名','自治体コード'],as_index=False).mean().sort_values(['年','自治体コード'])
# voters_q = voters_tmp[~(voters_tmp.duplicated())]
# 当選落選者の選挙結果のデータにしているので重複しています、重複を削除。
#voters_q.to_excel('./ss.xlsx',index=False)
voters_q
#voters_q.sort_values(['年','自治体コード']).reset_index(drop=True).drop(columns='自治体コード') # 結果表示

,年,区名,自治体コード,定数,候補者数,有権者数,投票率%,投票者数,得票率%
0,2011,中央,131024,30.0,39.0,97887.0,45.59,44626.6833,2.494359
1,2011,台東,131067,32.0,39.0,143834.0,49.57,71298.5138,2.476923
2,2011,品川,131091,40.0,55.0,296585.0,40.58,120354.1930,1.786727
5,2011,目黒,131105,35.0,55.0,213142.0,38.96,83040.1232,1.786364
3,2011,大田,131113,50.0,73.0,561414.0,43.29,243036.1206,1.325479
4,2011,渋谷,131130,34.0,46.0,168344.0,40.22,67707.9568,2.115435
6,2015,中央,131024,30.0,48.0,110176.0,45.64,50284.3264,2.020208
7,2015,台東,131067,32.0,42.0,151090.0,44.15,66706.2350,2.340238
8,2015,品川,131091,40.0,54.0,302599.0,40.08,121281.6792,1.772364
11,2015,目黒,131105,36.0,49.0,219196.0,39.35,86253.6260,2.001429


In [22]:
print(list(voters_q[voters_q['年']==2011]['区名']))

['中央', '台東', '品川', '目黒', '大田', '渋谷']


### 2. 過去3回の区議会選挙における、主要政党の得票数

In [22]:
#party_name = ele_par[~(ele_par['略称党名'] == '諸派')][['略称党名','党順序']]
# 諸派を除く政党の実績を上記データから抽出

import numpy as np # 計算ライブラリを読み込む

#pv = ele_result.pivot_table(index=['略称党名','党順序','年'], 
#                            columns='区名', 
#                            aggfunc=sum, 
#                            values=['得票数'])
# ピボット集計する。政党、年で各政党がどれくらい得票したか
group_columns = ['年','自治体コード','区名','略称党名','党順序'])
pv = ele_result.groupby(.sum().sort_values(['党順序'])

pv

#v = pv.loc[['自民','民主','立憲','国民','共産','社民','公明','維新','NHK'],:].applymap('{:,.0f}'.format)
#v.reset_index(level='党順序',drop=True)
#　データを整形、実績ある政党の数値を抽出


得票数   年齢   定数  候補者数      有権者数  投票率%      投票者数  得票率%
年    自治体コード 区名 略称党名 党順序                                                       
2015 131105 目黒 自民   1   27,897  880  612   833 3,726,332   669 1,466,312    32
     131067 台東 自民   1   21,653  647  384   504 1,813,080   530   800,475    32
2011 131130 渋谷 自民   1   17,717  608  374   506 1,851,784   442   744,788    26
2015 131091 品川 自民   1   31,827  672  520   702 3,933,787   521 1,576,662    26
2019 131130 渋谷 自民   1   17,441  597  408   660 2,215,224   480   885,204    24
...                        ...  ...  ...   ...       ...   ...       ...   ...
     131024 中央 諸派   99   2,009  160  120   176   512,000   183   233,677     3
2011 131091 品川 諸派   99  12,334  348  320   440 2,372,680   325   962,834    10
2019 131067 台東 諸派   99   4,090  102   64    82   320,590    85   136,475     6
2011 131113 大田 諸派   99  38,190  573  600   876 6,736,968   519 2,916,433    16
2019 131130 渋谷 諸派   99   4,509  165  136   220   738,408   160   295,068     6

[135 rows x 8 columns]

### 3. 過去3回の区議会選挙における、当選者のみ主要政党の得票数

In [34]:
import numpy as np # 計算ライブラリを読み込む

qv = ele_result[ele_result['当落']=='当'].pivot_table(index=['略称党名','党順序','年'], 
                                                   columns='区名',
                                                   aggfunc=sum,
                                                   values=['得票数'])
# ピボット集計する。当選議員のみ、政党、年で各政党がどれくらい得票したか

qv = qv.loc[['自民','民主','立憲','国民','共産','公明','維新','NHK'],:].applymap('{:,.0f}'.format)
qv.reset_index(level='党順序',drop=True)
#　データを整形、実績ある政党の数値を抽出


得票数                                        
区名             中央      台東      品川      大田      渋谷      目黒
略称党名 年                                                   
自民   2011  14,572  17,171  28,621  67,792  13,690  24,317
     2015  16,104  19,908  28,621  69,658  17,724  23,121
     2019  19,536  14,895  30,143  62,707  14,417  19,877
民主   2011   1,725  12,420  11,180  22,505   9,702   8,360
     2015   1,143   9,914  11,180  18,817   8,339   6,537
立憲   2019   4,278   3,679   6,069  15,796   9,755  13,295
国民   2019     nan   4,663   1,994     nan     nan     nan
共産   2011   3,970   7,323  15,633  31,713   8,632   7,142
     2015   5,125   8,086  15,633  37,031  10,616  11,685
     2019   2,396   7,564  16,280  28,146   5,984   9,323
公明   2011   6,079  10,099  20,031  45,924   9,172  10,669
     2015   5,263   9,313  20,031  41,479   8,500  10,163
     2019   5,702   9,580  18,634  46,702   7,855  11,226
維新   2015   3,080   1,167     nan   8,893     nan   3,917
     2019   1,108     nan   2,844   5,448     nan   2,048
NHK  2019   1,100   1,656   2,532   3,036   1,428   1,979

### 4. 全得票、当選議員のみ得票でどれくらい差があるか？

In [39]:
#現在絶賛処理中

pvv = pv
pvv=pvv.reset_index(drop=False)
pvv.columns = pvv.columns.droplevel(0)

qvv = qv
qvv=qvv.reset_index(drop=False)
qvv.columns = qvv.columns.droplevel(0)